In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F

import pynvml
import psutil

import cv2
import numpy as np

import matplotlib.pyplot as plt


import os
import sys

from tqdm import tqdm
from IPython.display import clear_output

sys.path.append('/home/meribejayson/Desktop/Projects/SharkCNN/training_models/YOLO3D/yolo_dataloaders')
sys.path.append("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/YOLO3D/model_impl")

from train_dataset import SharkYOLODataset
from yolo3d import YOLO3D
from yolo_loss import YOLO_Loss

In [ ]:
torch.manual_seed(12)

if not torch.cuda.is_available():
    raise Exception("Couldn't find CUDA")

device = torch.device("cuda")

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

In [ ]:
num_frames = 50
first_conv_layer_size = 16

In [ ]:
class YOLO_Loss(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.high = YOLO_Loss()
        self.mid = YOLO_Loss()
        self.low = YOLO_Loss()

    def forward(self, high, high_hat, mid, mid_hat, low, low_hat):
        total_loss = 0

        xbox_hat, xclass_hat = high_hat
        xbox, xclass , num_pos_exps = high

        total_loss += self.high(xbox, xbox_hat, xclass, xclass_hat, num_pos_exps)

        xbox_hat, xclass_hat = mid_hat
        xbox, xclass , num_pos_exps = mid

        total_loss += self.high(xbox, xbox_hat, xclass, xclass_hat, num_pos_exps)

        xbox_hat, xclass_hat = low_hat
        xbox, xclass , num_pos_exps = low

        total_loss += self.high(xbox, xbox_hat, xclass, xclass_hat, num_pos_exps)
        
        return total_loss

In [ ]:
shark_dataset = SharkYOLODataset(num_frames)
data_loader = data.DataLoader(shark_dataset, batch_size=1, num_workers=1)

In [ ]:
# state_dict = torch.load("./train-1/ann_weights_train_1.tar")

In [ ]:
model = YOLO3D(num_frames, num_features=first_conv_layer_size)
# model.load_state_dict(state_dict)
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0001)

In [ ]:
loss_module = YOLO_Loss()

In [ ]:
LARGE_NUM = 2e120
target_loss_change = 1e-6

def train_model(model, optimizer, data_loader, epochs=25):
    model.train()
    last_average_loss = LARGE_NUM
    curr_average_loss = 0

    for epoch in tqdm(range(epochs), desc='Epoch Progress'):
        total_iter_avg_loss = 0

        # Wrapping data_loader with tqdm for visibility on each example/batch
        for point in tqdm(data_loader, desc=f'Processing Data - Epoch {epoch+1}', leave=False):
            exp, (low, mid, high) = point
            low_pred, mid_pred, high_pred = model(exp)

            loss = loss_module(high, high_pred, mid, mid_pred, low, low_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_iter_avg_loss += loss.item()

        if (epoch % 3) == 0:
            state_dict = model.state_dict()
            torch.save(state_dict, f"/yolo3d_{epoch}.tar")

        last_average_loss = curr_average_loss
        # Assuming 'data_loader' accurately represents the number of examples processed
        curr_average_loss = total_iter_avg_loss / len(data_loader)  

        print(f'Epoch: {epoch}, Last Average Loss: {last_average_loss:.6f}, Current Average Loss: {curr_average_loss:.6f}')
        # Assuming setup for pynvml and psutil is configured
        # print(f"CPU Usage: {psutil.cpu_percent()}% GPU memory usage: {int(info.used / info.total * 100)}% \n")

In [ ]:
train_model(model, optimizer, data_loader)

In [ ]:
pynvml.nvmlShutdown()